In [4]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)
from selenium import webdriver
from time import sleep
import datetime

In [86]:
def mk_test_df(race_id):
    #出馬表サイトへ
    url1 = "http://race.sp.netkeiba.com/?pid=shutuba_past&race_id=" +str(race_id)
    r1 = requests.get(url1)
    soup = BeautifulSoup(r1.text.encode(r1.encoding))

    #馬のリスト
    HorseList = soup.select('.HorseList')

    #頭数
    headCount = int(soup.select('.Race_Data')[0].get_text().split('頭')[0].split(')')[1].replace(' ', ''))

    #################
    #頭数分のtest_df作成##
    #################
    test_df=pd.read_csv('/Users/wasedafuuta/Desktop/keiba/test_df_template/mk_feature_df.csv')
    test_df = test_df[0:headCount]
    
    #型指定
    is_object = ['grade', 'surface' ,'course', 'placeCode' , 'horse_number' ,'sex' ,'preOOF' , 'pre2OOF' ,]
    test_df[is_object] = test_df[is_object].astype(object)

    is_float = ['distance', 'month' ,'headCount' , 'weight' ,'age' ,'enterTimes' ,'eps','winRun' ,'disRoc' , 'odds',
               'preLastPhase' , 'preHeadCount' , 'surfaceChanged'  , 'gradeChanged' , 'owinper' ,'jwinper' ,'jEps' ,'twinper',
                'femaleOnly','ridingStrongJockey']
    test_df[is_float] = test_df[is_float].astype(float)


    
    #race_id
    test_df['race_id'] = race_id
    
    #grade
    if soup.select('.RaceDetail')[0].select('p') == []:
        grade = 0   
    else:
        raceName = soup.select('.RaceDetail')[0].select('p')[0].get_text()
        grade = 0
        if raceName.find('オープン') > -1 :
            grade = 0
        elif raceName.find('1600万下') > -1:
            grade = 1
        elif raceName.find('1000万下') > -1:
            grade = 2
        elif raceName.find('500万下') > -1:
            grade = 3
        elif raceName.find('未勝利') > -1:
            grade = 4
        elif raceName.find('新馬') > -1:
            grade = 5
    test_df['grade'] = grade

    #surface
    surface = soup.select('.Race_Data')[0].select('span')[0].get_text()[0]
    test_df['surface'] = surface

    #distance
    distance = float(soup.select('.Race_Data')[0].select('span')[0].get_text()[1:-1])
    test_df['distance'] = distance

    #course
    course = soup.select('.Race_Data')[0].get_text().split('(')[1].split(')')[0]
    test_df['course'] = course

    #month
    month =  soup.select('.Day')[0].string.replace('\n','').split('/')[0]
    test_df['month'] = float(month)

    #placeCode
    if soup.select('.Course')==[]:
        placeCode = ''
    else:
        placeCode = soup.select('.Course')[0].string.replace('\n','')
    test_df['placeCode'] = placeCode

    #headCount
    test_df['headCount'] = float(headCount)
    
    #####################
    ##各馬について、データ収集##
    #####################
    for n in range(headCount):  
    #horse_id　出馬表から
        #馬nのurl
        HorseHtml = HorseList[n]
        HorseUrl = HorseHtml.select('.Horse_Info')[0].select('.Horse02')[0].select('a')[0].get('href')

        #馬nのHorseId取得
        param = HorseUrl.split('&')
        HorseId = 0
        #用意した配列にパラメータを格納
        for i in range(len(param)):
            paramItem = param[i].split('=');
            if paramItem[0] == 'id':
                HorseId  = paramItem[1]
    
    #馬の競走成績サイト
        url = 'http://db.netkeiba.com/horse/'+HorseId+'/' 
        r = requests.get(url)
        HorseData_Soup = BeautifulSoup(r.text.encode(r.encoding))
        
        #馬のプロフィール
        HorseData_Prof = HorseData_Soup.select('.db_prof_table')[0]
        #過去競走成績データベースボックス
        HorseData_Result = HorseData_Soup.select('.db_h_race_results')[0].select('tbody')[0]


    #馬主ID取得
        OwnerId  = HorseData_Prof.select('a')[1].get('href').split('/')[2]

    #馬主のプロフィールサイト
        url4 = 'http://db.sp.netkeiba.com/?pid=owner_profile&id=' + OwnerId
        r4 = requests.get(url4)
        Owner_Soup = BeautifulSoup(r4.text.encode(r4.encoding))

    #調教師のurl
        TrainerUrl = HorseHtml.select('.Horse_Info')[0].select('.Horse05')[0].select('a')[0].get('href')

        #馬０のTrainerId取得
        param = TrainerUrl.split('&')
        TrainereId = 0
        #用意した配列にパラメータを格納
        for i in range(len(param)):
            paramItem = param[i].split('=');
            if paramItem[0] == 'id':
                TrainerId  = paramItem[1]

    #調教師のプロフィールサイト
        url5 = 'http://db.netkeiba.com/trainer/result/' + str(TrainerId) + '/'
        r5 = requests.get(url5)
        Trainer_Soup = BeautifulSoup(r5.text.encode(r5.encoding))

    #騎手のurl
        JockeyUrl = HorseHtml.select('.Jockey')[0].select('a')[0].get('href')

        #馬０のJockeyId取得
        param = JockeyUrl.split('&')
        JockeyId = 0
        #用意した配列にパラメータを格納
        for i in range(len(param)):
            paramItem = param[i].split('=');
            if paramItem[0] == 'id':
                JockeyId  = paramItem[1]

        # 騎手のプロフィールサイト
        url6 = 'http://db.sp.netkeiba.com/?pid=jockey_profile&id=' + JockeyId
        r6 = requests.get(url6)
        Jockey_Soup = BeautifulSoup(r6.text.encode(r6.encoding))
        
        #horse_id , Jockey_id , Trainer_id
        test_df['horse_id'][n] = HorseId
        test_df['jockey_id'][n] = JockeyId
        test_df['trainer_id'][n] = TrainerId


        #horse_number
        test_df.at[n,'horse_number'] = n+1

        #weight
        weight = float(soup.select('.Jockey')[n].select('span')[1].get_text())
        test_df.at[n,'weight'] = weight

        #sex , age
        sex = soup.select('.Barei')[n].string[0]
        age = float(soup.select('.Barei')[n].string[1])
        test_df.at[n,'sex'] = sex
        test_df.at[n,'age'] = age

        #enterTimes
        enterTimes = float(HorseData_Prof.get_text().split('通算成績')[1].split('戦')[0].replace('\n',''))
        test_df.at[n,'enterTimes'] = enterTimes

        #eps(平均賞金)
        if enterTimes >= 1 :
            racelen = len(HorseData_Result.select('tr'))
            shokin_Sum = 0
            for i in range(racelen):
                shokin = HorseData_Result.select('tr')[i].select('td')[-1].string.replace(',','')
                if shokin == '\xa0':
                    shokin = 0
                else :
                    shokin = float(shokin)
                shokin_Sum += shokin
            eps = shokin_Sum/enterTimes
            test_df.at[n,'eps'] = eps
        else : 
            test_df.at[n,'eps'] = 0

        #disRoc
        if enterTimes >= 1 :
            racelen = len(HorseData_Result.select('tr'))
            dist_sum = 0.0
            for i in range(racelen):
                dist = HorseData_Result.select('tr')[i].select('td')[14].string[1:]
                dist_sum  +=  float(dist)
            mean = dist_sum/enterTimes
            disRoc = (distance-mean)/mean
            test_df.at[n,'disRoc'] = disRoc
        else:
            test_df.at[n,'disRoc'] = np.nan
        
        #winRun 勝率
        orders = HorseData_Prof.get_text().split('[')[1].split(']')[0].split('-')
        winRun = float(orders[0]) /enterTimes
        test_df.at[n,'winRun'] = winRun

        #owinper
        if len(Owner_Soup.select('.SP_Common_Table')[1].select('td')) >=5 :
            owinper = float(Owner_Soup.select('.SP_Common_Table')[1].select('td')[5].string.replace('%',''))/100
            test_df.at[n,'owinper'] = owinper
        else :
            test_df.at[n,'owinper'] = 0

        #jwinper
        if len(Jockey_Soup.select('.SP_Common_Table')[1].select('td'))>=5 :
            jwinper = float(Jockey_Soup.select('.SP_Common_Table')[1].select('td')[5].string.replace('%',''))/100
            test_df.at[n,'jwinper'] = jwinper
            #jEps
            all_prize = float(Jockey_Soup.select('.DB_Data_Right')[1].get_text().replace('\n',
                                                                                         '').replace('収得賞金','').replace(' ','').replace('円', '').replace(',',  ''))/10000
            jEnterTimes  = float(Jockey_Soup.select('.SP_Common_Table')[1].select('td')[4].string.replace(',',''))
            jEps = all_prize/jEnterTimes
            test_df.at[n,'jEps'] = jEps
        else:
            test_df.at[n,'jwinper'] = 0
            test_df.at[n,'jEps'] = 0

        #twinper
        twinper = float(Trainer_Soup.select('tr')[2].select('td')[16].string)
        test_df.at[n,'twinper'] = twinper
        
        #############
        ##前走について##
        #############
        if enterTimes >= 1:
            #前走レースId取得
            preRaceId = HorseData_Result.select('a')[2].get('href').replace('race','').replace('/','')

            #前走レースページへ
            url7 = 'http://db.sp.netkeiba.com/v0/?pid=race_detail&id=' + preRaceId
            r7 = requests.get(url7)
            preRace_Soup = BeautifulSoup(r7.text.encode(r7.encoding))
            
            #preOOF
            preOOF = HorseData_Result.select('tr')[0].select('td')[11].string
            test_df.at[n,'preOOF'] = preOOF

            #preLastPhase
            fault_list = ['(降)',
                                    '(再)',
                                      '中',
                                      '取',
                                      '失',
                                      '除']
            fault = 0
            for f in fault_list:
                if preOOF.find(f) > -1:
                    fault = 1
                    break
            if fault == 0 and len(preRace_Soup.select('.Race_Common_Table_Box tr'))>=5:
                preLastPhase = preRace_Soup.select('.Race_Common_Table_Box tr')[int(preOOF)].select('td')[11].get_text()
                test_df.at[n,'preLastPhase'] = preLastPhase
            else:
                test_df.at[n,'preLastPhase'] = np.nan

            #preHeadCount
            preHeadCount  =  HorseData_Result.select('tr')[0].select('td')[6].string
            test_df.at[n,'preHeadCount'] = preHeadCount

            #surfaceChanged
            preSurface = HorseData_Result.select('tr')[0].select('td')[14].string[0]
            surfaceChanged = 0
            if preSurface != surface :
                surfaceChanged = 1
            test_df.at[n,'surfaceChanged'] = surfaceChanged

            #gradeChanged
            preRaceName = preRace_Soup.select('.Race_Header_Info')[0].select('dd')[1].string
            preGrade = 0
            if preRaceName.find('オープン') > -1 :
                preGrade = 0
            elif preRaceName.find('1600万下') > -1:
                preGrade = 1
            elif preRaceName.find('1000万下') > -1:
                preGrade = 2
            elif preRaceName.find('500万下') > -1:
                preGrade = 3
            elif preRaceName.find('未勝利') > -1:
                preGrade = 4
            elif preRaceName.find('新馬') > -1:
                preGrade = 5
            gradeChanged = 0
            if grade != preGrade:
                gradeChanged = 1
            test_df.at[n,'gradeChanged'] = gradeChanged  
            
            #RidingStrongJockey
            preJockeyId = HorseData_Result.select('tr')[0].select('td')[12].select('a')[0].get('href').split('/')[-2]
            if preJockeyId!=JockeyId:
                    url = 'http://db.sp.netkeiba.com/?pid=jockey_profile&id=' + str(preJockeyId)
                    r = requests.get(url)
                    preJockey_Soup = BeautifulSoup(r.text.encode(r.encoding))
                    prej_all_prize = float(preJockey_Soup.select('.DB_Data_Right')[1].get_text().replace('\n',
                                                                                                 '').replace('収得賞金','').replace(' ','').replace('円', '').replace(',',  ''))/10000
                    prejEnterTimes  = float(preJockey_Soup.select('.SP_Common_Table')[1].select('td')[4].string.replace(',',''))
                    prejEps = prej_all_prize/prejEnterTimes
                    if prejEps < jEps:
                        test_df.at[n, 'ridingStrongJockey']= 1.0
                    else:
                        test_df.at[n, 'ridingStrongJockey']= 0.0
            
        else:
            test_df.at[n,'preOOF'] = np.nan
            test_df.at[n,'preLastPhase'] = np.nan
            test_df.at[n,'preHeadCount'] = np.nan
            test_df.at[n,'surfaceChanged'] = np.nan
            test_df.at[n,'gradeChanged'] = np.nan  
            test_df.at[n, 'ridingStrongJockey'] = np.nan
            
            
        #pre2OOF
        if  enterTimes >=2:
            pre2OOF = HorseData_Result.select('tr')[1].select('td')[11].string
            test_df.at[n,'pre2OOF'] = pre2OOF
        else:
            test_df.at[n,'pre2OOF'] = np.nan


        print(n)
    
    
    #femaleOnly
    if len(test_df['sex'].unique())==1 and (test_df['sex'].unique()[0] == '牝') +0 ==1:
        test_df['femaleOnly'] = 1

    return test_df

In [6]:
odds = []
hweight = []
dhweight = []
def put_pdds_weight_dweight(test_df, odds,hweight,dhweight):
    if len(odds)!=len(hweight) or len(odds)!=len(dhweight):
        print('配列の長さが違います。確認してください。')
    else:
        for n in range(len(odds)):
            test_df.at[n,'odds'] = float(odds[n])
            test_df.at[n,'hweight'] = float(hweight[n])
            test_df.at[n,'dhweight'] = float(dhweight[n])   
    return test_df

In [7]:
weather = ''
def put_weather(test_df , weather):
    test_df['weather'] = weather
    return test_df

In [8]:
test_df.to_csv('test_df'+str(race_id)+'.csv' ,index=False)

NameError: name 'test_df' is not defined

In [96]:
#長良川
race_id = '201807040412'
test_df = mk_test_df(race_id)
test_df

/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:152: SettingWithCopyWarning: 
A value is trying 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


,race_id,horse_number,grade,order_of_finish,horse_id,jockey_id,trainer_id,age,avgsr4,avgWin4,dhweight,disavesr,disRoc,distance,dsi,enterTimes,eps,hweight,jwinper,odds,owinper,preSRa,sex,surface,surfaceScore,twinper,weather,weight,winRun,jEps,jAvgWin4,preOOF,pre2OOF,month,ridingStrongJockey,runningStyle,preLastPhase,lateStartPer,course,placeCode,headCount,preHeadCount,surfaceChanged,gradeChanged,preMargin,femaleOnly
0,201807040412,1,0,0,2013100376,1117,1008,5.0,0,0,0,0,0.198594,2200.0,0,31.0,59.658065,0,0.056,0.0,0.065,0,牡,芝,0,0.064,0,57.0,0.000000,99.392810,0,8,9,12.0,1.0,0,34.5,0,左,中京,15.0,15.0,0.0,0.0,0,0.0
1,201807040412,2,0,0,2014102300,894,427,4.0,0,0,0,0,0.200000,2200.0,0,18.0,86.111111,0,0.082,0.0,0.083,0,牡,芝,0,0.094,0,57.0,0.111111,162.510785,0,5,4,12.0,1.0,0,35.2,0,左,中京,15.0,16.0,0.0,0.0,0,0.0
2,201807040412,3,0,0,2015101185,1168,1107,3.0,0,0,0,0,-0.035088,2200.0,0,10.0,62.500000,0,0.046,0.0,0.123,0,牡,芝,0,0.079,0,55.0,0.100000,62.126353,0,11,14,12.0,0.0,0,37.8,0,左,中京,15.0,16.0,0.0,0.0,0,0.0
3,201807040412,4,0,0,2015105076,5243,1020,3.0,0,0,0,0,0.157895,2200.0,0,2.0,400.000000,0,0.058,0.0,0.116,0,牡,芝,0,0.068,0,55.0,0.500000,110.403427,0,5,1,12.0,1.0,0,34.9,0,左,中京,15.0,10.0,0.0,0.0,0,0.0
4,201807040412,5,0,0,2014103163,1034,1078,4.0,0,0,0,0,0.091603,2200.0,0,13.0,204.246154,0,0.044,0.0,0.080,0,牡,芝,0,0.062,0,57.0,0.153846,95.920598,0,5,11,12.0,0.0,0,34.5,0,左,中京,15.0,17.0,0.0,0.0,0,0.0
5,201807040412,6,0,0,2015105026,1166,1002,3.0,0,0,0,0,0.042105,2200.0,0,9.0,85.000000,0,0.064,0.0,0.126,0,牡,芝,0,0.121,0,55.0,0.111111,84.143296,0,5,9,12.0,0.0,0,35.0,0,左,中京,15.0,14.0,0.0,0.0,0,0.0
6,201807040412,7,0,0,2014101302,1144,1150,4.0,0,0,0,0,0.075188,2200.0,0,13.0,86.453846,0,0.065,0.0,0.060,0,牡,芝,0,0.042,0,57.0,0.076923,103.760620,0,9,11,12.0,0.0,0,35.2,0,左,中京,15.0,16.0,0.0,0.0,0,0.0
7,201807040412,8,0,0,2015105008,1038,1075,3.0,0,0,0,0,0.241935,2200.0,0,7.0,71.428571,0,0.040,0.0,0.080,0,牡,芝,0,0.095,0,55.0,0.142857,72.112339,0,7,1,12.0,1.0,0,34.7,0,左,中京,15.0,17.0,0.0,0.0,0,0.0
8,201807040412,9,0,0,2015104868,1152,1061,3.0,0,0,0,0,0.250000,2200.0,0,5.0,208.000000,0,0.059,0.0,0.108,0,牡,芝,0,0.142,0,55.0,0.200000,78.956694,0,2,1,12.0,0.0,0,35.6,0,左,中京,15.0,9.0,0.0,0.0,0,0.0
9,201807040412,10,0,0,2015101077,1157,1157,3.0,0,0,0,0,0.073171,2200.0,0,8.0,139.975000,0,0.059,0.0,0.088,0,牡,芝,0,0.075,0,55.0,0.125000,99.860231,0,2,9,12.0,0.0,0,35.9,0,左,中京,15.0,13.0,0.0,0.0,0,0.0


In [97]:
odds = [18.7,
117.3,
6.8,
3.9,
11.4,
39.6,
24.8,
5.4,
4.1,
7.3,
49.9,
41.9,
86.7,
320.9
       ]
hweight = [486,
486,
504,
498,
504,
426,
496,
482,
482,
504,
418,
462,
472,
434
          ]
dhweight = [8,2,24,-2,-4,0,-2,14,-2,6,-24,2,0,-8]
test_df = put_pdds_weight_dweight(test_df, odds,hweight,dhweight)
test_df = put_weather(test_df , '晴')
test_df.to_csv('test_df'+str(201807040412)+'.csv',index=False)

In [52]:
#名古屋日刊
race_id = '201807040411'
test_df = mk_test_df(race_id)
test_df

/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:145: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:146: SettingWithCopyWarning: 
A value is trying 

0
1
2
3
4
5
6
7
8
9
10
11


,race_id,horse_number,grade,order_of_finish,horse_id,jockey_id,trainer_id,age,avgsr4,avgWin4,dhweight,disavesr,disRoc,distance,dsi,enterTimes,eps,hweight,jwinper,odds,owinper,preSRa,sex,surface,surfaceScore,twinper,weather,weight,winRun,jEps,jAvgWin4,preOOF,pre2OOF,month,ridingStrongJockey,runningStyle,preLastPhase,lateStartPer,course,placeCode,headCount,preHeadCount,surfaceChanged,gradeChanged,preMargin,femaleOnly
0,201807040411,1,0,0,2014105125,1168,1033,4.0,0,0,0,0,-0.084507,2000.0,0,13.0,232.023077,0,0.046,0.0,0.068,0,牡,芝,0,0.131,0,57.0,0.153846,62.126353,0,7,5,12.0,0.0,0,35.4,0,左,中京,12.0,9.0,0.0,0.0,0,0.0
1,201807040411,2,0,0,2015101701,5268,1148,3.0,0,0,0,0,0.000000,2000.0,0,12.0,247.933333,0,0.019,0.0,0.118,0,牝,芝,0,0.131,0,54.0,0.166667,38.489765,0,6,1,12.0,0.0,0,32.9,0,左,中京,12.0,10.0,0.0,0.0,0,0.0
2,201807040411,3,0,0,2013106154,1095,1164,5.0,0,0,0,0,0.283422,2000.0,0,24.0,189.479167,0,0.079,0.0,0.110,0,牝,芝,0,0.156,0,55.0,0.083333,138.281896,0,4,2,12.0,0.0,0,33.8,0,左,中京,12.0,10.0,0.0,0.0,0,0.0
3,201807040411,4,0,0,2012110063,1144,1067,6.0,0,0,0,0,0.147132,2000.0,0,23.0,201.182609,0,0.065,0.0,0.082,0,牡,芝,0,0.199,0,57.0,0.130435,103.760620,0,8,3,12.0,1.0,0,35.2,0,左,中京,12.0,13.0,0.0,0.0,0,0.0
4,201807040411,5,0,0,2013101799,1157,1103,5.0,0,0,0,0,-0.030303,2000.0,0,24.0,189.650000,0,0.059,0.0,0.102,0,セ,芝,0,0.159,0,57.0,0.125000,99.860231,0,6,6,12.0,0.0,0,33.3,0,左,中京,12.0,9.0,0.0,0.0,0,0.0
5,201807040411,6,0,0,2013105476,1112,1128,5.0,0,0,0,0,0.004016,2000.0,0,25.0,88.800000,0,0.041,0.0,0.098,0,牡,芝,0,0.166,0,57.0,0.080000,66.164355,0,7,9,12.0,0.0,0,35.0,0,左,中京,12.0,8.0,0.0,0.0,0,0.0
6,201807040411,7,0,0,2013100900,1034,425,5.0,0,0,0,0,-0.008264,2000.0,0,36.0,134.652778,0,0.044,0.0,0.074,0,牡,芝,0,0.171,0,57.0,0.083333,95.920598,0,11,6,12.0,0.0,0,35.5,0,左,中京,12.0,11.0,0.0,0.0,0,0.0
7,201807040411,8,0,0,2012103708,1128,1150,6.0,0,0,0,0,-0.138991,2000.0,0,35.0,236.071429,0,0.066,0.0,0.065,0,牡,芝,0,0.098,0,57.0,0.085714,104.480336,0,3,5,12.0,1.0,0,33.9,0,左,中京,12.0,10.0,0.0,0.0,0,0.0
8,201807040411,9,0,0,2015104679,1166,1144,3.0,0,0,0,0,0.351351,2000.0,0,10.0,270.860000,0,0.064,0.0,0.082,0,牝,芝,0,0.166,0,54.0,0.200000,84.143296,0,1,15,12.0,1.0,0,33.9,0,左,中京,12.0,13.0,0.0,0.0,0,0.0
9,201807040411,10,0,0,2015110075,1172,427,3.0,0,0,0,0,0.428571,2000.0,0,1.0,0.000000,0,0.005,0.0,0.125,0,牝,芝,0,0.187,0,54.0,0.000000,16.410256,0,16,NaN,12.0,0.0,0,38.7,0,左,中京,12.0,18.0,0.0,1.0,0,0.0


In [53]:
odds = [22.7,
        6.3,
        4.5,
       26.0,
       15.9,
       101.1,
       37.7,
       11.6,
       9.6,
        141.7,
        2.7,
        7.7
       ]
hweight = [510,
           496,
           444,
           488,
           450,
           442,
           516,
           502,
           454,
           516,
           522,
           474
          ]
dhweight = [-4,10,6,2,-6,0,0,0,0,10,0,2]
test_df = put_pdds_weight_dweight(test_df, odds,hweight,dhweight)
test_df = put_weather(test_df , '晴')
test_df.to_csv('test_df'+str(201807040411)+'.csv',index=False)

In [48]:
#堺Ｓ
race_id = '201809050410'
test_df = mk_test_df(race_id)
test_df

/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:145: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:146: SettingWithCopyWarning: 
A value is trying 

0
1
2
3
4
5
6
7
8


,race_id,horse_number,grade,order_of_finish,horse_id,jockey_id,trainer_id,age,avgsr4,avgWin4,dhweight,disavesr,disRoc,distance,dsi,enterTimes,eps,hweight,jwinper,odds,owinper,preSRa,sex,surface,surfaceScore,twinper,weather,weight,winRun,jEps,jAvgWin4,preOOF,pre2OOF,month,ridingStrongJockey,runningStyle,preLastPhase,lateStartPer,course,placeCode,headCount,preHeadCount,surfaceChanged,gradeChanged,preMargin,femaleOnly
0,201809050410,1,0,0,2015101632,1037,1044,3.0,0,0,0,0,0.067797,1800.0,0,7.0,432.228571,0,0.052,0.0,0.113,0,牡,ダ,0,0.199,0,56.0,0.428571,94.898762,0,11,1,12.0,0.0,0,38.6,0,右,阪神,9.0,15.0,0.0,0.0,0,0.0
1,201809050410,2,0,0,2012104401,5386,1128,6.0,0,0,0,0,0.011236,1800.0,0,15.0,389.213333,0,0.144,0.0,0.098,0,牡,ダ,0,0.166,0,57.0,0.266667,268.616096,0,7,4,12.0,1.0,0,36.4,0,右,阪神,9.0,16.0,0.0,0.0,0,0.0
2,201809050410,3,0,0,2011101337,1127,1003,7.0,0,0,0,0,-0.106742,1800.0,0,53.0,201.843396,0,0.070,0.0,0.032,0,牡,ダ,0,0.131,0,57.0,0.075472,114.668713,0,7,8,12.0,1.0,0,37.1,0,右,阪神,9.0,9.0,0.0,0.0,0,0.0
3,201809050410,4,0,0,2013104166,1015,1065,5.0,0,0,0,0,0.174935,1800.0,0,25.0,180.172000,0,0.054,0.0,0.080,0,牡,ダ,0,0.122,0,57.0,0.120000,99.337803,0,8,13,12.0,0.0,0,36.9,0,右,阪神,9.0,9.0,0.0,0.0,0,0.0
4,201809050410,5,0,0,2014103298,1154,1158,4.0,0,0,0,0,0.014085,1800.0,0,16.0,498.568750,0,0.070,0.0,0.087,0,牡,ダ,0,0.139,0,57.0,0.250000,108.588213,0,2,2,12.0,0.0,0,36.5,0,右,阪神,9.0,9.0,0.0,0.0,0,0.0
5,201809050410,6,0,0,2014102137,1014,438,4.0,0,0,0,0,0.227273,1800.0,0,12.0,793.825000,0,0.129,0.0,0.113,0,牝,ダ,0,0.222,0,55.0,0.333333,267.333421,0,3,2,12.0,1.0,0,36.3,0,右,阪神,9.0,16.0,0.0,0.0,0,0.0
6,201809050410,7,0,0,2013100904,1093,1155,5.0,0,0,0,0,-0.012539,1800.0,0,35.0,315.271429,0,0.082,0.0,0.032,0,牝,ダ,0,0.145,0,55.0,0.114286,165.373437,0,9,4,12.0,0.0,0,38.2,0,右,阪神,9.0,9.0,0.0,0.0,0,0.0
7,201809050410,8,0,0,2015104384,5473,1110,3.0,0,0,0,0,-0.037433,1800.0,0,11.0,421.881818,0,0.146,0.0,0.113,0,牡,ダ,0,0.135,0,56.0,0.181818,301.909519,0,7,3,12.0,1.0,0,35.4,0,右,阪神,9.0,10.0,1.0,0.0,0,0.0
8,201809050410,9,0,0,2013110103,1116,1062,5.0,0,0,0,0,-0.027027,1800.0,0,26.0,257.676923,0,0.071,0.0,0.067,0,牡,ダ,0,0.203,0,57.0,0.153846,126.400140,0,取,16,12.0,0.0,0,NaN,0,右,阪神,9.0,15.0,0.0,0.0,0,0.0


In [49]:
odds = [6.9,
        40.3,
        68.6,
       43.2,
       7.7,
       1.4,
       27.7,
       5.5,
       54.0
       ]
hweight = [524,
           526,
           554,
           500,
           526,
           488,
           506,
           532,
           494]
dhweight = [10,
            12,
            10,
            -4,
            2,
            0,
            -6,
            2,
            8]
test_df = put_pdds_weight_dweight(test_df, odds,hweight,dhweight)
test_df = put_weather(test_df , '晴')
test_df.to_csv('test_df'+str(201809050410)+'.csv',index=False)